## Идея
Парсим полный проект (нужные файлы), добавляем их в контекст. Запускаем запрос в GPT на исправление (что хотим получить).
Можем сохранять идеи по форматированию, но это не обязательно.

In [156]:
import os
import logging
from typing import List, Dict, Any
from dotenv import load_dotenv, find_dotenv
import tiktoken
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.schema import Document
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableConfig


a = load_dotenv(find_dotenv())  # read local .env file
opena_api_key = os.environ['OPENAI_API_KEY']

In [146]:
def num_tokens_from_content(content: str, model: str = "gpt-3.5-turbo") -> int:
    """
    Calculate the number of tokens in a given content string using a specific encoding model.

    Parameters
    ----------
    content : str
        The text content for which to calculate the number of tokens.
    model : str, optional
        The name of the model whose encoding is to be used for tokenization. Defaults to "gpt-3.5-turbo".

    Returns
    -------
    int
        The number of tokens in the content as determined by the encoding of the specified model.

    Notes
    -----
    The function uses the 'tiktoken' library to obtain the encoding specific to the provided model. It then 
    calculates and returns the number of tokens that the content would be split into according to that encoding. 
    In case of an unrecognized model, it defaults to the encoding for "cl100k_base".
    """
    try:
        # Assuming 'tiktoken' is a hypothetical library for tokenization
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(content))
    return num_tokens

### Пробегаемся по директориям
Составляем список того, что будет лежать в папке, со всеми путями

In [147]:
import os
from typing import List, Dict

def read_files(folder_path: str, target_extensions: List[str], always_include_files: List[str], excluded_directories: List[str]) -> List[Dict[str, str]]:
    """
    Recursively read files from a given directory and its subdirectories, 
    filtering by file extensions, including specified always include files, 
    and excluding specified directories.

    Parameters
    ----------
    folder_path : str
        The root directory from which to start reading files.
    target_extensions : list of str
        List of file extensions to include in the search (e.g., ['.py', '.txt']).
    always_include_files : list of str
        List of file names to always include in the final output regardless of their extensions or directories.
    excluded_directories : list of str
        List of directory names to exclude from the search.

    Returns
    -------
    list of dict
        A list of dictionaries, each dictionary contains 'path' (the relative path of the file), 
        'filename' (the name of the file), and 'content' (the content of the file).

    Notes
    -----
    The function reads the content of each file that matches the target extensions or is listed in the 
    always include files, excluding files in any of the excluded directories. It's designed to be used 
    for text files as it reads contents into a string.

    """
    files_list = []
    for subdir, dirs, files in os.walk(folder_path):
        dirs[:] = [d for d in dirs if d not in excluded_directories]
        for file in files:
            full_path = os.path.join(subdir, file)
            if file.endswith(tuple(target_extensions)) or file in always_include_files:
                with open(full_path, 'r', encoding="utf-8") as f:
                    files_list.append({
                        'path': os.path.relpath(full_path, folder_path),
                        'filename': file,
                        'content': f.read()
                    })
    return files_list


In [148]:
folder_path = 'D:/9-GitHubR/gh_telegram_scrapper'
target_extensions = ['.py','.sql',]
always_include_files = ["Dockerfile","docker-compose.yml"]
excluded_directories = ['.vscode','.venv','app_streamlit']

file_dict = read_files(folder_path, target_extensions, always_include_files, excluded_directories)

print(len(file_dict))
[x['path'] for x in file_dict]

11


['docker-compose.yml',
 'Dockerfile',
 'app\\__init__.py',
 'app\\src\\main.py',
 'app\\src\\__init__.py',
 'app\\src\\utils\\db.py',
 'app\\src\\utils\\image_handler.py',
 'app\\src\\utils\\tg_client.py',
 'app\\src\\utils\\__init__.py',
 'app\\tests\\__init__.py',
 'sql\\init.sql']

## Добавляем статистику по токенам, сколько чего будет

In [149]:
def augment_file_data(files_data: List[Dict[str, str]]) -> List[Dict[str, Any]]:
    """
    Process a list of file dictionaries, reading content from each and augmenting the dictionary
    with the length of content, the number of words, and the number of tokens.

    Parameters
    ----------
    files_data : list of dict
        A list of dictionaries, each containing 'path', 'filename', and 'content' keys.

    Returns
    -------
    list of dict
        The same list of dictionaries, but each dictionary is augmented with 'length' (number of characters),
        'words' (number of words), and 'tokens' (number of tokens using num_tokens_from_content) fields.

    Notes
    -----
    This function assumes that the content of each file is text and uses the 'num_tokens_from_content' function
    to calculate the number of tokens according to a specific model's encoding.

    """
    for file_dict in files_data:
        content = file_dict['content']
        file_dict['length'] = len(content)  # Number of characters
        file_dict['words'] = len(content.split())  # Number of words
        file_dict['tokens'] = num_tokens_from_content(content)  # Number of tokens

    return files_data

In [150]:
stat_file_dict = augment_file_data(file_dict)

print ("Total tokens:", sum([x["tokens"] for x in stat_file_dict]))
[(x["path"], x["tokens"]) for x in stat_file_dict]

Total tokens: 3224


[('docker-compose.yml', 302),
 ('Dockerfile', 85),
 ('app\\__init__.py', 0),
 ('app\\src\\main.py', 1062),
 ('app\\src\\__init__.py', 0),
 ('app\\src\\utils\\db.py', 740),
 ('app\\src\\utils\\image_handler.py', 73),
 ('app\\src\\utils\\tg_client.py', 624),
 ('app\\src\\utils\\__init__.py', 0),
 ('app\\tests\\__init__.py', 0),
 ('sql\\init.sql', 338)]

## Грузим контент всего проекта в промпт, и задаем вопросы

In [136]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


In [151]:
# logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.CRITICAL)


In [186]:
folder_path = 'D:/9-GitHubR/gh_telegram_scrapper'
target_extensions = ['.py','.sql',]
always_include_files = ["Dockerfile","docker-compose.yml"]
excluded_directories = ['.vscode','.venv','app_streamlit']

file_dict = read_files(folder_path, target_extensions, always_include_files, excluded_directories)


docs = [
    Document(
        page_content = item["content"],
        metadata={"path": item["path"]}
        )
    for item in file_dict
]

# ----------------------------------------------------

# old ------------------------------------
# model_name = "gpt-3.5-turbo-16k"
# current --------------------------------
# Input costs || Output costs
# model_name = "gpt-3.5-turbo-1106" # supports a 16K context | 	$0.0010 / 1K tokens ||	$0.0020 / 1K tokens
# model_name = "gpt-3.5-turbo-instruct" #  supports a 4K context | $0.0015 / 1K tokens ||	$0.0020 / 1K tokens
model_name = "gpt-4-1106-preview" # With 128k context, 4k - output | $0.01 / 1K tokens || $0.03 / 1K tokens
# model_name = "gpt-4" # With 4k context |$0.03 / 1K tokens ||$0.06 / 1K tokens
# model_name = "gpt-4-32k" # With 32k context |$0.06 / 1K tokens ||	$0.12 / 1K tokens

file_template_str = """
LOCAL FILEPATH: {path}
CONTENTS:
{page_content}
"""

doc_prompt = PromptTemplate.from_template(file_template_str)

# ----------------------------------------------------
question = """
Поправь пожалуйста в текущем проекте в блоке main. Сейчас в одном блоке try сразу несколько функций объединено, в частности, когда "пробегаемся по всем сообщениям", то у нас идет: 
* Сохраняем attachments
* Обработка и сохранение ответов на сообщения
и в случае ошибки например с сохранением reply у нас цикл сбрасывается на следующее сообщение.

Но по сути это 2 разные логические штуки, возможно стоит их по отдельности обрабатывать.

Может быть вообще этот try разбить на несколько


"""

# Поправь их пожалуйста. От полноты и точности твоего ответа зависит моя карьера. Я дам тебе премию в 100 USD в случае успешного исправления.
# Опиши кратко структуру представленного проекта. Что проект делает? Из каких компонетов он состоит? Какие есть особенности (что сохраняет, куда именно)?

# ----------------------------------------------------
system_prompt = """You are a high-end python and posgresql programmer.
You don't teach how to write programs, you write them.
Отвечай на русском языке."""

# prompt_template = PromptTemplate.from_template(
#     "Context:\n\n{content}\n\nQuestion:\n\n{question}\n\n"
# )

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","{system}"),
        ("user","Context:\n\n{content}"),
        ("user","Question:\n\n{question}"),
    ]
)

# Создание цепочки
chain = (
    {
        # Создание контента из документов
        "content": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        ),
        # Добавление вопроса
        "question": lambda _:question
        # Добавление вопроса
        ,
        "system": lambda _:system_prompt
    }
    # | prompt_template  # Применение шаблона к контенту и вопросу
    | chat_prompt_template  # Применение шаблона к контенту и вопросу
    | ChatOpenAI(model_name=model_name, temperature=0.0, verbose=True)
    | StrOutputParser()  # Преобразование вывода модели в строку
)



# result = chain.invoke(docs)
with get_openai_callback() as cb:
    result = chain.invoke(docs, {"callbacks": [cb]})



print(
    f"Total Tokens: {cb.total_tokens}\n"\
    f"Prompt Tokens: {cb.prompt_tokens}\n"\
    f"Completion Tokens: {cb.completion_tokens}\n"\
    f"Total Cost (USD): ${cb.total_cost}"\
)
print("="*40)

# pprint.pprint(chain.invoke(docs).text)
# print(chain.invoke(docs).text)
print(result)

Total Tokens: 7157
Prompt Tokens: 6332
Completion Tokens: 825
Total Cost (USD): $0.0
Для улучшения обработки ошибок и разделения логики, мы можем разбить большой блок `try` на несколько меньших, каждый из которых будет отвечать за свою часть логики. Это позволит нам более точно управлять поведением программы в случае возникновения исключений и не прерывать обработку всех сообщений из-за ошибки в одном из них.

Вот пример того, как можно разделить блок `try` в вашем коде:

```python
for dialog in dialogs:
    dialog_type: str = dialog.get("_", " ")
    dialog_id: int = dialog.get("id", 0)
    dialog_title: str = dialog.get("title", " ")

    try:
        logger.debug(f"Processing channel: {dialog_title}")
        self.database.check_and_add_channel(
            dialog_id, dialog_title, dialog_type
        )
    except Exception as e:
        logger.error(f"Error while adding channel {dialog_title}: {e}")
        continue

    try:
        logger.info(f"Fetching messages from {dialog_tit

: 

## Вопрос по конкретному файлу

In [180]:
folder_path = 'D:/9-GitHubR/gh_telegram_scrapper'
target_extensions = []
always_include_files = ["db.py"]
excluded_directories = ['.vscode','.venv','app_streamlit']

file_dict = read_files(folder_path, target_extensions, always_include_files, excluded_directories)

docs = [
    Document(
        page_content = item["content"],
        metadata={"path": item["path"]}
        )
    for item in file_dict
]

# ----------------------------------------------------

# old ------------------------------------
# model_name = "gpt-3.5-turbo-16k"
# current --------------------------------
# Input costs || Output costs
# model_name = "gpt-3.5-turbo-1106" # supports a 16K context | 	$0.0010 / 1K tokens ||	$0.0020 / 1K tokens
# model_name = "gpt-3.5-turbo-instruct" #  supports a 4K context | $0.0015 / 1K tokens ||	$0.0020 / 1K tokens
model_name = "gpt-4-1106-preview" # With 128k context, 4k - output | $0.01 / 1K tokens || $0.03 / 1K tokens
# model_name = "gpt-4" # With 4k context |$0.03 / 1K tokens ||$0.06 / 1K tokens
# model_name = "gpt-4-32k" # With 32k context |$0.06 / 1K tokens ||	$0.12 / 1K tokens

file_template_str = """
LOCAL FILEPATH: {path}
CONTENTS:
{page_content}
"""

doc_prompt = PromptTemplate.from_template(file_template_str)

# ----------------------------------------------------
question = """


"""
# Добавь обработку исключений: В некоторых местах кода возможны исключительные ситуации, но они не обрабатываются.
# Рекомендуется добавить соответствующие блоки try-except для обработки исключений и вывода информации об ошибках.
# - В файле attachment_handler.py в методе `save_attachment` класса `AttachmentHandler` можно добавить обработку исключений, которые могут возникнуть при сохранении файлов, например `OSError`.


# Опиши кратко структуру представленного проекта. Что проект делает? Из каких компонетов он состоит? Какие есть особенности (что сохраняет, куда именно)?

# ----------------------------------------------------
system_prompt = """You are a high-end python and posgresql programmer.
You don't teach how to write programs, you write them.
Return full script (I don't have fingers).
Return only full script.
"""

# prompt_template = PromptTemplate.from_template(
#     "Context:\n\n{content}\n\nQuestion:\n\n{question}\n\n"
# )

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","{system}"),
        ("user","Context:\n\n{content}"),
        ("user","Question:\n\n{question}"),
    ]
)

# Создание цепочки
chain = (
    {
        # Создание контента из документов
        "content": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        ),
        # Добавление вопроса
        "question": lambda _:question
        # Добавление вопроса
        ,
        "system": lambda _:system_prompt
    }
    # | prompt_template  # Применение шаблона к контенту и вопросу
    | chat_prompt_template  # Применение шаблона к контенту и вопросу
    | ChatOpenAI(model_name=model_name, temperature=0.0, verbose=True)
    | StrOutputParser()  # Преобразование вывода модели в строку
)



# result = chain.invoke(docs)
with get_openai_callback() as cb:
    result = chain.invoke(docs, {"callbacks": [cb]})



print(
    f"Total Tokens: {cb.total_tokens}\n"\
    f"Prompt Tokens: {cb.prompt_tokens}\n"\
    f"Completion Tokens: {cb.completion_tokens}\n"\
    f"Total Cost (USD): ${cb.total_cost}"\
)
print("="*40)

# pprint.pprint(chain.invoke(docs).text)
# print(chain.invoke(docs).text)
print(result)

Total Tokens: 2284
Prompt Tokens: 1621
Completion Tokens: 663
Total Cost (USD): $0.0
Чтобы добавить обработку других типов вложений (документы, видео и т.д.) в текущий проект, вам нужно будет расширить класс `Database` в модуле `db.py`, добавив методы для обработки и сохранения этих типов вложений. Вам также потребуется обновить схему базы данных, чтобы она могла хранить информацию о новых типах вложений.

Ниже приведен пример того, как можно расширить класс `Database` для обработки документов и видео:

```python
class Database:
    # ... существующие методы ...

    def add_document_attachment(self, message_id: int, dialog_id: int, document):
        """Add a document attachment to the Attachments table.

        Args:
            message_id (int): The ID of the message the document belongs to.
            dialog_id (int): The ID of the dialog the document belongs to.
            document: The document object from Telethon.
        """
        attachment_type_id = self.add_attachment_